In [ ]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.134, 0.140, 0.160, 0.140],
    [0.199, 0.105, 0.304, 0.104],
    [0.156, 0.093, 0.061, 0.084],
    [0.021, 0.128, 0.050, 0.128],
    [0.103, 0.185, 0.041, 0.206],
    [0.178, 0.152, 0.157, 0.116],
    [0.021, 0.128, 0.069, 0.091],
    [0.188, 0.070, 0.160, 0.131]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


          C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  Rank
Site2  0.199  0.105  0.304  0.104      0.397515       0.063050  0.334465     1
Site6  0.178  0.152  0.157  0.116      0.216882       0.053892  0.162990     2
Site1  0.134  0.140  0.160  0.140      0.185836       0.069101  0.116735     3
Site8  0.188  0.070  0.160  0.131      0.150103       0.173410 -0.023308     4
Site5  0.103  0.185  0.041  0.206      0.156478       0.191241 -0.034762     5
Site7  0.021  0.128  0.069  0.091      0.050072       0.214571 -0.164499     6
Site3  0.156  0.093  0.061  0.084      0.038842       0.229942 -0.191101     7
Site4  0.021  0.128  0.050  0.128      0.037619       0.238141 -0.200522     8
